In [13]:
from geoband.API import *

In [52]:
import folium
import json
import geopandas as gpd

import pandas as pd
import numpy as np

import json

import seaborn as sns

In [30]:
# 데이터를 서버 주피터 노트북에 다운
GetCompasData('SBJ_2012_001', '8', '8.오산시_유동인구(2019).csv')
GetCompasData('SBJ_2012_001', '9', '9.오산시_어린이보호구역.csv')
GetCompasData('SBJ_2012_001', '31', '31.오산시_법정경계(시군구).geojson')
GetCompasData('SBJ_2012_001', '33', '33.오산시_법정경계(읍면동).geojson')

In [28]:
zone=pd.read_csv("9.오산시_어린이보호구역.csv")
print(zone.shape)
zone.head()

(91, 7)


,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도
0,특수학교,성심학교,N,0,25,127.017521,37.176762
1,초등학교,가수초등학교,Y,1,12,127.047957,37.148103
2,초등학교,광성초등학교,Y,2,12,127.039709,37.184197
3,초등학교,대호초등학교,Y,3,12,127.057889,37.159106
4,초등학교,매홀초등학교,Y,2,6,127.063282,37.162749


In [29]:
ppl=pd.read_csv("8.오산시_유동인구(2019).csv")
print(ppl.shape)
ppl.head()

(125163, 27)


,STD_YM,lon,lat,TMST_00,TMST_01,TMST_02,TMST_03,TMST_04,TMST_05,TMST_06,...,TMST_14,TMST_15,TMST_16,TMST_17,TMST_18,TMST_19,TMST_20,TMST_21,TMST_22,TMST_23
0,201901,126.995145,37.176703,0.08,0.05,0.04,0.04,0.08,0.27,0.80,...,0.69,0.76,0.95,0.91,0.70,0.43,0.34,0.22,0.14,0.10
1,201901,126.995142,37.177153,0.06,0.04,0.03,0.03,0.06,0.19,0.56,...,0.54,0.58,0.69,0.66,0.51,0.33,0.26,0.17,0.11,0.08
2,201901,126.995714,37.175804,0.00,0.00,0.00,0.00,0.00,0.01,0.03,...,0.04,0.04,0.05,0.05,0.03,0.02,0.02,0.01,0.01,0.00
3,201901,126.995711,37.176254,0.05,0.03,0.02,0.03,0.05,0.19,0.56,...,0.48,0.53,0.65,0.63,0.48,0.29,0.23,0.15,0.10,0.07
4,201901,126.995708,37.176705,0.02,0.01,0.01,0.01,0.02,0.07,0.18,...,0.19,0.20,0.23,0.22,0.17,0.11,0.09,0.06,0.04,0.03


In [35]:
ssg=gpd.read_file('31.오산시_법정경계(시군구).geojson')
emd=gpd.read_file('33.오산시_법정경계(읍면동).geojson')

In [37]:
ssg.head()

,SIG_CD,SIG_KOR_NM,geometry
0,41370,오산시,"MULTIPOLYGON (((127.02723 37.20044, 127.02736 ..."


In [39]:
ssg['geometry']

0    MULTIPOLYGON (((127.02723 37.20044, 127.02736 ...
Name: geometry, dtype: geometry

In [38]:
emd.head()

,EMD_CD,EMD_KOR_NM,geometry
0,41370101,오산동,"MULTIPOLYGON (((127.08320 37.16169, 127.08313 ..."
1,41370102,부산동,"MULTIPOLYGON (((127.08334 37.16167, 127.08357 ..."
2,41370103,원동,"MULTIPOLYGON (((127.07977 37.14891, 127.08002 ..."
3,41370104,궐동,"MULTIPOLYGON (((127.04708 37.16834, 127.04783 ..."
4,41370105,청학동,"MULTIPOLYGON (((127.05794 37.15723, 127.05815 ..."


In [57]:
zone['시설종류'].value_counts()

어린이집    48
초등학교    25
유치원     17
특수학교     1
Name: 시설종류, dtype: int64

In [65]:
zone_cmap={'어린이집':'orange', 
           '초등학교':'green', 
           '유치원':'blue', 
           '특수학교':'red'}
zone['marker_color']=zone['시설종류'].map(zone_cmap)

In [66]:
zone.head()

,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도,marker_color
0,특수학교,성심학교,N,0,25,127.017521,37.176762,red
1,초등학교,가수초등학교,Y,1,12,127.047957,37.148103,green
2,초등학교,광성초등학교,Y,2,12,127.039709,37.184197,green
3,초등학교,대호초등학교,Y,3,12,127.057889,37.159106,green
4,초등학교,매홀초등학교,Y,2,6,127.063282,37.162749,green


In [71]:
# 오산시 중심 맵 생성
center = [37.14887, 127.07728]
m = folium.Map(location=center, zoom_start=11)
folium.GeoJson(ssg).add_to(m)
m

In [76]:
# 어린이보호구역 시각화
for i in range(len(zone)):
    folium.Marker(
    location = zone.loc[i, ['보호구역_위도', '보호구역_경도']],
    popup = zone.loc[i, '시설명'],
    radius = 15,
    icon=folium.Icon(color=zone.loc[i, 'marker_color'], icon='child', prefix='fa')
    ).add_to(m)

m

In [47]:
ppl

,STD_YM,lon,lat,TMST_00,TMST_01,TMST_02,TMST_03,TMST_04,TMST_05,TMST_06,...,TMST_14,TMST_15,TMST_16,TMST_17,TMST_18,TMST_19,TMST_20,TMST_21,TMST_22,TMST_23
0,201901,126.995145,37.176703,0.08,0.05,0.04,0.04,0.08,0.27,0.80,...,0.69,0.76,0.95,0.91,0.70,0.43,0.34,0.22,0.14,0.10
1,201901,126.995142,37.177153,0.06,0.04,0.03,0.03,0.06,0.19,0.56,...,0.54,0.58,0.69,0.66,0.51,0.33,0.26,0.17,0.11,0.08
2,201901,126.995714,37.175804,0.00,0.00,0.00,0.00,0.00,0.01,0.03,...,0.04,0.04,0.05,0.05,0.03,0.02,0.02,0.01,0.01,0.00
3,201901,126.995711,37.176254,0.05,0.03,0.02,0.03,0.05,0.19,0.56,...,0.48,0.53,0.65,0.63,0.48,0.29,0.23,0.15,0.10,0.07
4,201901,126.995708,37.176705,0.02,0.01,0.01,0.01,0.02,0.07,0.18,...,0.19,0.20,0.23,0.22,0.17,0.11,0.09,0.06,0.04,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125158,201912,127.097213,37.154107,0.09,0.07,0.05,0.04,0.04,0.04,0.05,...,0.11,0.12,0.12,0.12,0.14,0.14,0.14,0.14,0.12,0.11
125159,201912,127.097819,37.145997,0.23,0.18,0.12,0.09,0.09,0.11,0.14,...,0.28,0.28,0.30,0.34,0.34,0.35,0.35,0.34,0.32,0.28
125160,201912,127.097793,37.150955,0.02,0.02,0.02,0.02,0.02,0.02,0.02,...,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04
125161,201912,127.098354,37.151407,0.02,0.02,0.02,0.00,0.00,0.00,0.02,...,0.02,0.02,0.02,0.02,0.04,0.04,0.04,0.04,0.02,0.02
